# Duygu Analizi (Sentiment Analysis) - Makine Öğrenmesi

Bu not defterinde, kendi oluşturduğumuz küçük bir veri seti üzerinde makine öğrenmesi yöntemleri kullanarak duygu analizi yapacağız.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

## 1. Veri Seti Oluşturma
Örnek olarak küçük bir Türkçe veri seti oluşturuyoruz.

In [ ]:
data = {
    'text': [
        'Bu film harika', 'Çok beğendim', 'Mükemmel bir ürün', 'Kesinlikle tavsiye ederim', 'Çok güzel',
        'Hiç beğenmedim', 'Berbat bir deneyim', 'Zaman kaybı', 'Çok kötü', 'Tavsiye etmem',
        'Oyunculuklar şahaneydi', 'Senaryo çok sürükleyici', 'Görüntü kalitesi muazzam',
        'Sıkıcı bir filmdi', 'Param boşa gitti', 'Beklentimi karşılamadı'
    ],
    'label': [
        'Pozitif', 'Pozitif', 'Pozitif', 'Pozitif', 'Pozitif',
        'Negatif', 'Negatif', 'Negatif', 'Negatif', 'Negatif',
        'Pozitif', 'Pozitif', 'Pozitif',
        'Negatif', 'Negatif', 'Negatif'
    ]
}

df = pd.DataFrame(data)
print("Veri Seti:")
print(df)

## 2. Veri Ön İşleme ve Vektörleştirme
Metin verilerini makine öğrenmesi modelinin anlayabileceği sayısal formata çeviriyoruz (Bag of Words modeli).

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

print("Öznitelik İsimleri (Kelime Hazinesi):")
print(vectorizer.get_feature_names_out())

## 3. Model Eğitimi
Veriyi eğitim ve test olarak ayırıp Naive Bayes modeli ile eğitiyoruz.

In [ ]:
# Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Eğitimi (Multinomial Naive Bayes)
model = MultinomialNB()
model.fit(X_train, y_train)

# Test Seti Üzerinde Değerlendirme
y_pred = model.predict(X_test)
print(f"\nModel Doğruluğu: {accuracy_score(y_test, y_pred)}")

## 4. Yeni Veri ile Tahmin
Modeli daha önce görmediği cümlelerle test ediyoruz.

In [ ]:
yeni_yorumlar = ["Bu ürünü çok sevdim", "Hiç güzel değil", "Fena değil ama daha iyi olabilirdi", "Tam bir hayal kırıklığı"]
yeni_vektor = vectorizer.transform(yeni_yorumlar)
tahminler = model.predict(yeni_vektor)

print("\nYeni Yorumlar ve Tahminler:")
for yorum, tahmin in zip(yeni_yorumlar, tahminler):
    print(f"Yorum: '{yorum}' -> Tahmin: {tahmin}")

## 5. RNN (Recurrent Neural Network) ile Duygu Analizi
Aynı veri seti üzerinde Derin Öğrenme yöntemi olan RNN kullanarak model eğiteceğiz.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM

# Parametreler
vocab_size = 1000 # Kelime hazinesi büyüklüğü
embedding_dim = 16
max_length = 20 # Maksimum cümle uzunluğu
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [ ]:
# Tokenization (Metni sayısal dizilere çevirme)
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df['text'])
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print("Padded Sequences:")
print(padded)

In [ ]:
# Etiketleri hazırlama (Pozitif=1, Negatif=0)
labels = np.array([1 if label == 'Pozitif' else 0 for label in df['label']])

# Eğitim ve Test ayırma
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [ ]:
# RNN Modelini Oluşturma
model_rnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

In [ ]:
# Modeli Eğitme
history = model_rnn.fit(X_train_rnn, y_train_rnn, epochs=50, validation_data=(X_test_rnn, y_test_rnn), verbose=1)

In [ ]:
# Test etme
loss, accuracy = model_rnn.evaluate(X_test_rnn, y_test_rnn)
print(f"RNN Model Doğruluğu: {accuracy}")

## 6. LSTM (Long Short-Term Memory) ile Duygu Analizi
RNN'in gelişmiş bir versiyonu olan LSTM kullanarak model eğiteceğiz. LSTM, uzun vadeli bağımlılıkları öğrenmede daha başarılıdır.

In [ ]:
# LSTM Modelini Oluşturma
model_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

In [ ]:
# Modeli Eğitme
history_lstm = model_lstm.fit(X_train_rnn, y_train_rnn, epochs=50, validation_data=(X_test_rnn, y_test_rnn), verbose=1)

In [ ]:
# Test etme
loss, accuracy = model_lstm.evaluate(X_test_rnn, y_test_rnn)
print(f"LSTM Model Doğruluğu: {accuracy}")